In [1]:
import pandas as pd
import datetime as dt
import random as rnd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
elo_df = pd.read_csv(r'data\elo_data_set.csv', parse_dates=[0])
teams = []
for i in elo_df['team'].unique(): #constructs list of team names
    teams.append(i)
elo_df.head()

,datetime,team,season,opp,day_week,week,W/L,ot,record,home/away,...,defense_expected_points,sp_expected_points,team_elo_before_all,team_elo_after_all,opp_elo_before_all,opp_elo_after_all,team_elo_before_season,team_elo_after_season,opp_elo_before_season,opp_elo_after_season
0,2002-09-05 20:38:00,NYG,2002,SFO,Thu,1,0.0,0.0,0-1,HOME,...,-0.79,3.46,1500.0,1485.0,1500.0,1515.0,1500.0,1480.0,1500.0,1520.0
1,2002-09-08 13:04:00,CAR,2002,BAL,Sun,1,1.0,0.0,1-0,HOME,...,6.68,4.20,1500.0,1515.0,1500.0,1485.0,1500.0,1520.0,1500.0,1480.0
2,2002-09-08 13:04:00,CHI,2002,MIN,Sun,1,1.0,0.0,1-0,HOME,...,-1.20,1.01,1500.0,1515.0,1500.0,1485.0,1500.0,1520.0,1500.0,1480.0
3,2002-09-08 13:04:00,CLE,2002,KAN,Sun,1,0.0,0.0,0-1,HOME,...,-17.94,1.15,1500.0,1485.0,1500.0,1515.0,1500.0,1480.0,1500.0,1520.0
4,2002-09-08 13:04:00,BUF,2002,NYJ,Sun,1,0.0,1.0,0-1,HOME,...,0.59,-7.81,1500.0,1485.0,1500.0,1515.0,1500.0,1480.0,1500.0,1520.0


In [3]:
# separates out played games, and the upcoming week.
#season_df = elo_df[elo_df['season']==dt.datetime.today()]
season_df = elo_df[elo_df['season']==dt.datetime.today().year]

history_df = elo_df[elo_df['datetime'] < dt.datetime.today()]

future_df = season_df[season_df['datetime'] >= dt.datetime.today()]


upcoming_week = history_df['week'].iloc[-1] + 1
future_df = future_df[future_df['week'] == upcoming_week]
future_df = future_df[['datetime', 'team', 'opp', 'team_score', 'opp_score', 'team_elo_before_all', 'opp_elo_before_all', 'team_elo_before_season', 'opp_elo_before_season', 'W/L']]
print(f'Week to predict : {upcoming_week}')

Week to predict : 10


In [4]:
# Generates a dictionary of dataframes for points gained/allowed per team. This allows convenient generation of the distributions fed into the game 
team_data_dict_all = {}
team_data_dict_season = {}
for i in range(len(teams)):
    team = teams[i]
    team_data_all = pd.DataFrame()
    team_data_season = pd.DataFrame()
    
    team_off_all = []
    team_def_all = []
    team_elo_all = []
    
    team_off_season = []
    team_def_season = []
    team_elo_season = []
    
    team_df_all = history_df[(history_df['team'] == f'{team}') | (history_df['opp'] == f'{team}')]
    team_df_season = season_df[(season_df['team'] == f'{team}') | (season_df['opp'] == f'{team}')]
    team_df_all.reset_index(inplace=True)
    team_df_all.drop('index', axis=1, inplace=True)
    
    team_df_season.reset_index(inplace=True)
    team_df_season.drop('index', axis=1, inplace=True)
    
    for x in range(len(team_df_all)):
        if team_df_all['team'][x] == f'{team}':
            team_off_all.append(team_df_all['team_score'][x])
            team_def_all.append(team_df_all['opp_score'][x])
            team_elo_all.append(team_df_all['team_elo_before_all'][x])
        else:
            team_off_all.append(team_df_all['opp_score'][x])
            team_def_all.append(team_df_all['team_score'][x])
            team_elo_all.append(team_df_all['opp_elo_before_all'][x])
            
    for y in range(len(team_df_season)):
        if team_df_all['team'][y] == f'{team}':
            team_off_season.append(team_df_season['team_score'][y])
            team_def_season.append(team_df_season['opp_score'][y])
            team_elo_season.append(team_df_season['team_elo_before_all'][y])
        else:
            team_off_season.append(team_df_season['opp_score'][y])
            team_def_season.append(team_df_season['team_score'][y])
            team_elo_season.append(team_df_season['opp_elo_before_all'][y])
    
    team_data_all['team_off'] = team_off_all
    team_data_all['team_def'] = team_def_all
    team_data_all['elo'] = team_elo_all
    team_data_dict_all[team] = team_data_all
    
    team_data_season['team_off'] = team_off_season
    team_data_season['team_def'] = team_def_season
    team_data_season['elo'] = team_elo_season
    team_data_season.dropna(inplace=True)
    team_data_dict_season[team] = team_data_season
      
   
print(len(team_data_dict_all.keys()), ' ' , len(team_data_dict_season.keys()))

C:\Users\fitzp\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


32   32


In [5]:
# Games to be predicted
future_df.reset_index(inplace=True)
future_df.drop('index', axis=1, inplace=True)
future_df['team_elo_prediction_all%'] = np.nan
future_df['opp_elo_prediction_all%'] = np.nan
future_df['team_gauss_prediction_all%'] = np.nan
future_df['opp_gauss_prediction_all%'] = np.nan
future_df['tie_gauss_prediction_all%'] = np.nan

future_df['team_elo_prediction_season%'] = np.nan
future_df['opp_elo_prediction_season%'] = np.nan
future_df['team_gauss_prediction_season%'] = np.nan
future_df['opp_gauss_prediction_season%'] = np.nan
future_df['tie_gauss_prediction_season%'] = np.nan

future_df

,datetime,team,opp,team_score,opp_score,team_elo_before_all,opp_elo_before_all,team_elo_before_season,opp_elo_before_season,W/L,team_elo_prediction_all%,opp_elo_prediction_all%,team_gauss_prediction_all%,opp_gauss_prediction_all%,tie_gauss_prediction_all%,team_elo_prediction_season%,opp_elo_prediction_season%,team_gauss_prediction_season%,opp_gauss_prediction_season%,tie_gauss_prediction_season%
0,2020-11-12 20:20:00,TEN,IND,NaN,NaN,1573.785284,1498.704297,1427.384672,1434.526591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-15 13:00:00,DET,WAS,NaN,NaN,1396.059333,1350.999465,1502.888738,1490.117887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-15 13:00:00,CLE,HOU,NaN,NaN,1399.611075,1492.607616,1444.324490,1500.084340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-15 13:00:00,GNB,JAX,NaN,NaN,1625.133577,1332.700280,1467.511978,1443.972763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-15 13:00:00,NYG,PHI,NaN,NaN,1315.960617,1546.886707,1455.856339,1431.128320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-11-15 13:00:00,CAR,TAM,NaN,NaN,1417.051959,1499.226027,1439.937536,1469.725827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-11-15 16:05:00,ARI,BUF,NaN,NaN,1457.727023,1551.293114,1489.550040,1428.211783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-11-15 16:05:00,MIA,LAC,NaN,NaN,1457.248346,1425.303947,1481.002004,1475.172532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-11-15 16:05:00,LVR,DEN,NaN,NaN,1459.716219,1471.703406,1424.656570,1427.267262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-11-15 16:25:00,NOR,SFO,NaN,NaN,1664.458879,1542.958418,1410.131960,1481.131384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def color_predict(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0.5 else 'green'
    return 'color: %s' % color

In [7]:
# functions to simulate games

def elo_expected(teamA, teamB, dictionary):
    rating_A = dictionary[teamA]['elo'].iloc[-1]
    rating_B = dictionary[teamB]['elo'].iloc[-1]
    expected_A = 1 / (1 + 10 ** ((rating_B - rating_A) / 400 ))
    expected_B = 1 / (1 + 10 ** ((rating_A - rating_B) / 400 ))
    return [expected_A, expected_B]

def elo_game_sim(teamA, teamB, dictionary):
    win_values = elo_expected(teamA, teamB, dictionary)
    win_gen = rnd.random()
    if win_values[0] > win_values[1]:
        if win_gen < win_values[0]:
            return [1, 0]
        else:
            return [0, 1]
    if win_values[0] < win_values[1]:
        if win_gen < win_values[1]:
            return [0, 1]
        else:
            return[1, 0]

def elo_series_sim(teamA, teamB, runs, dictionary):
    games = []
    teamA_wins = 0
    teamB_wins = 0
    for i in range(runs):
        game = elo_game_sim(teamA, teamB, dictionary)
        games.append(game)
        if game[0] == 1:
            teamA_wins += 1
        else:
            teamB_wins += 1
    return [teamA_wins, teamB_wins]

def gauss_game_sim(teamA, teamB, dictionary):
    team_A_score = (rnd.gauss(dictionary[teamA]['team_off'].mean(), dictionary[teamA]['team_off'].std()) + rnd.gauss(dictionary[teamB]['team_def'].mean(), dictionary[teamB]['team_def'].std()))/2
    team_B_score = (rnd.gauss(dictionary[teamB]['team_off'].mean(), dictionary[teamB]['team_off'].std()) + rnd.gauss(dictionary[teamA]['team_def'].mean(), dictionary[teamA]['team_def'].std()))/2
    if int(round(team_A_score)) > int(round(team_B_score)):
        return 1
    elif int(round(team_A_score)) < int(round(team_B_score)):
        return -1
    else: 
        return 0

def gauss_series_sim(teamA, teamB, runs, dictionary):
    games = []
    teamA_wins = 0
    teamB_wins = 0
    ties = 0
    for i in range(runs):
        game = gauss_game_sim(teamA, teamB, dictionary)
        games.append(game)
        if game == 1:
            teamA_wins += 1
        elif game== -1:
            teamB_wins += 1
        else:
            gauss_game_sim(teamA, teamB, dictionary)
    return [teamA_wins, teamB_wins, ties]

In [ ]:
# simulations and future_df update
num_sims = 50000
for i in range(len(future_df)):
    teamA = future_df['team'].iloc[i]
    teamB = future_df['opp'].iloc[i]
    
    elo_result = elo_series_sim(teamA, teamB, num_sims, team_data_dict_all)
    elo_result_A = elo_result[0] / (elo_result[0] + elo_result[1])
    elo_result_B = elo_result[1] / (elo_result[0] + elo_result[1])
    future_df['team_elo_prediction_all%'].iloc[i] = elo_result_A
    future_df['opp_elo_prediction_all%'].iloc[i] = elo_result_B
    
    gauss_result = gauss_series_sim(teamA, teamB, num_sims, team_data_dict_all)
    gauss_result_A = gauss_result[0] / (gauss_result[0] + gauss_result[1] + gauss_result[2])
    gauss_result_B = gauss_result[1] / (gauss_result[0] + gauss_result[1] + gauss_result[2])
    gauss_result_tie = gauss_result[2] / (gauss_result[0] + gauss_result[1] + gauss_result[2])
    future_df['team_gauss_prediction_all%'].iloc[i] = gauss_result_A
    future_df['opp_gauss_prediction_all%'].iloc[i] = gauss_result_B
    future_df['tie_gauss_prediction_all%'].iloc[i] = gauss_result_tie
    
    elo_result_season = elo_series_sim(teamA, teamB, num_sims, team_data_dict_season)
    elo_result_A_season = elo_result_season[0] / (elo_result_season[0] + elo_result_season[1])
    elo_result_B_season = elo_result_season[1] / (elo_result_season[0] + elo_result_season[1])
    future_df['team_elo_prediction_season%'].iloc[i] = elo_result_A_season
    future_df['opp_elo_prediction_season%'].iloc[i] = elo_result_B_season
    
    gauss_result_season = gauss_series_sim(teamA, teamB, num_sims, team_data_dict_season)
    gauss_result_A_season = gauss_result_season[0] / (gauss_result_season[0] + gauss_result_season[1] + gauss_result_season[2])
    gauss_result_B_season = gauss_result_season[1] / (gauss_result_season[0] + gauss_result_season[1] + gauss_result_season[2])
    gauss_result_tie_season = gauss_result_season[2] / (gauss_result_season[0] + gauss_result_season[1] + gauss_result_season[2])
    future_df['team_gauss_prediction_season%'].iloc[i] = gauss_result_A_season
    future_df['opp_gauss_prediction_season%'].iloc[i] = gauss_result_B_season
    future_df['tie_gauss_prediction_season%'].iloc[i] = gauss_result_tie_season
    
    print('--------------------------------------------------------------')
    print(f'TEAMS : {teamA} | {teamB}')
    print(f'HISTORICAL PREDICTION: elo_result: {elo_result} | gaussian_result: {gauss_result}')
    print(f'SEASON PREDICTION: elo_result: {elo_result_season} | gaussian_result: {gauss_result_season}')

C:\Users\fitzp\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


--------------------------------------------------------------
TEAMS : TEN | IND
HISTORICAL PREDICTION: elo_result: [32874, 17126] | gaussian_result: [20049, 28092, 0]
SEASON PREDICTION: elo_result: [32884, 17116] | gaussian_result: [18570, 29460, 0]
--------------------------------------------------------------
TEAMS : DET | WAS
HISTORICAL PREDICTION: elo_result: [30579, 19421] | gaussian_result: [23086, 24812, 0]
SEASON PREDICTION: elo_result: [35508, 14492] | gaussian_result: [21609, 26104, 0]
--------------------------------------------------------------
TEAMS : CLE | HOU
HISTORICAL PREDICTION: elo_result: [18590, 31410] | gaussian_result: [20627, 27297, 0]
SEASON PREDICTION: elo_result: [13355, 36645] | gaussian_result: [17137, 31168, 0]
--------------------------------------------------------------
TEAMS : GNB | JAX
HISTORICAL PREDICTION: elo_result: [41127, 8873] | gaussian_result: [30480, 17611, 0]
SEASON PREDICTION: elo_result: [33754, 16246] | gaussian_result: [17036, 30851, 

In [ ]:
future_df

In [ ]:
predictions = []
predicted_teams = []
home_predict = []
opp_predict =[]

for i in range(len(future_df)):
    home = future_df["team_elo_prediction_all%"][i] + future_df["opp_gauss_prediction_all%"][i] + future_df["team_elo_prediction_season%"][i] + future_df["opp_elo_prediction_season%"][i]
    opp = future_df["opp_elo_prediction_all%"][i] + future_df["opp_gauss_prediction_all%"][i] + future_df["opp_elo_prediction_season%"][i] + future_df["opp_elo_prediction_season%"][i]
    
    if home > opp:
        predictions.append(0)
        predicted_teams.append(future_df['team'].iloc[i])
        home_predict.append(home)
        opp_predict.append(opp)
    else:
        predictions.append(1)
        predicted_teams.append(future_df['opp'].iloc[i])
        home_predict.append(home)
        opp_predict.append(opp)

future_df['home_sum_predict'] = home_predict
future_df['opp_sum_predict'] = opp_predict      
future_df['predicted_team'] = predicted_teams        
future_df['predicted_outcome'] = predictions



future_df.head(16)

In [ ]:
future_df.to_csv(f'data/predictions/{dt.datetime.today().date()}_week_{upcoming_week}_predictions.csv', index=False)

In [ ]:
name_list = []
for i in range(len(future_df['datetime'])):
    if future_df['predicted_outcome'].iloc[i] == 0:
        name_list.append(future_df['team'].iloc[i])
    else:
        name_list.append(future_df['opp'].iloc[i])
    
future_df[['home_sum_predict', 'opp_sum_predict']].plot(kind='bar', figsize=(12, 12))
locs, labels=plt.xticks()
plt.xticks(locs, name_list)
plt.savefig('current_predictions.png')
plt.title(f'Week {upcoming_week} Predictions')
plt.xlabel('Predicted Winner')
plt.show


In [ ]:
team1 = 'NOR'
team2 = 'TAM'

team_data_dict_all[team1]['team_off'].plot(kind='hist', figsize = (12,12), bins=21, alpha=0.5, label=team1)
team_data_dict_all[team2]['team_off'].plot(kind='hist', figsize = (12,12), bins=21, alpha=0.5, label=team2)
plt.axvline(team_data_dict_all[team1]['team_off'].mean(), linestyle=":", color='b', label = f'{team1} mean points scored')
plt.axvline(team_data_dict_all[team2]['team_off'].mean(), linestyle=':', color = 'r', label = f'{team2} mean points scored')
plt.xticks(range(0, 60, 3))
plt.legend()
plt.show()

In [ ]:
team_data_dict_all[team1]['team_def'].plot(kind='hist', figsize = (12,12), bins=21, alpha=0.5, label = team1)
team_data_dict_all[team2]['team_def'].plot(kind='hist', figsize = (12,12), bins=21, alpha=0.5, label = team2)
plt.axvline(team_data_dict_all[team1]['team_def'].mean(), linestyle=":", color='b', label = f'{team1} mean points allowed')
plt.axvline(team_data_dict_all[team2]['team_def'].mean(), linestyle=':', color = 'r', label = f'{team2} mean points allowed')
plt.xticks(range(0, 60, 3))
plt.legend()
plt.show()